In [1]:
import polars as pl
from datasets import load_dataset
import opendatasets as od

In [2]:
temp_df = None
final_df = None

def add_to_df(df, out_df = None):
    if out_df is None:
        out_df = df
    else:
        out_df = out_df.extend(df)
    return out_df

In [3]:
data_path = './supplementary-texts-rewritten-by-genai-models/test.csv'

df = pl.read_csv(data_path).select('original_text').unique().with_columns([
    pl.lit('supplementary-texts-rewritten-by-genai-models').alias('source')
])

temp_df = add_to_df(df, temp_df)

In [4]:
# https://huggingface.co/datasets/euclaise/writingprompts
ds = load_dataset('euclaise/writingprompts')
df = pl.from_arrow(ds['train'].data.table).select(
    pl.col('story').alias('original_text'),
    pl.lit('writingprompts').alias('source')
)
temp_df = add_to_df(df, temp_df)

In [5]:
# https://huggingface.co/datasets/Abirate/english_quotes
ds = load_dataset('Abirate/english_quotes')
df = pl.from_arrow(ds['train'].data.table).select(
    pl.col('quote').alias('original_text'),
    pl.lit('english_quotes').alias('source')
)
temp_df = add_to_df(df, temp_df)

In [6]:
# https://www.kaggle.com/datasets/aashita/nyt-comments

dataset_url = 'https://www.kaggle.com/datasets/aashita/nyt-comments'
ods = od.download(dataset_url)

ds_path = '/home/lawrence/Projects/contests/llm_prompt_recovery/nyt-comments/CommentsApril2018.csv'

df = pl.read_csv(ds_path).select(
    pl.col('commentBody').alias('original_text'),
    pl.lit('nyt-comments').alias('source')
)

temp_df = add_to_df(df, temp_df)

Skipping, found downloaded files in "./nyt-comments" (use force=True to force download)


In [7]:
temp_df = temp_df.with_columns([
    pl.col('original_text').str.len_chars().alias('og_text_len'),
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("og_text_id")
]).filter(
    pl.col('og_text_len') < 500
).unique('original_text')

In [8]:
for source in temp_df['source'].unique().to_list():
    if source == 'supplementary-texts-rewritten-by-genai-models':
        final_df = add_to_df(temp_df.filter(pl.col('source') == source).sample(fraction=1), final_df)
    elif source == 'writingprompts':
        final_df = add_to_df(temp_df.filter(pl.col('source') == source).sample(fraction=1), final_df)
    elif source == 'english_quotes':
        final_df = add_to_df(temp_df.filter(pl.col('source') == source).sample(n=1200), final_df)
    elif source == 'nyt-comments':
        final_df = add_to_df(temp_df.filter(pl.col('source') == source).sample(n=1200), final_df)

In [9]:
# final_df.unique('original_text').write_csv('data/original_texts.csv')

In [20]:
import spacy_fastlang
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("language_detector")

def get_lang(text: str) -> str:
    doc = nlp(text)
    return doc._.language



In [22]:
final_df = final_df.with_columns([
    pl.col("original_text").map_elements(get_lang).alias("lang"),
]).filter(pl.col('lang') == 'en') #.write_csv('data/original_texts.csv')

In [24]:
final_df.unique('original_text').write_csv('data/original_texts.csv')

In [58]:
train_df = pl.read_parquet("./data/train_data_2/*.parquet")


In [59]:
train_df[88,3]

"**Competency to be Acquired:**\n\nThe ability to advocate effectively for oneself and ensure the protection of one's well-being in the face of biased social workers who may prioritize personal biases over the best interests of the child."

In [66]:
train_df.filter(
    # pl.col('generated_text').str.contains('\*\*.*\*\*'),
    ~pl.col('generated_text').str.contains('I am unable')
).write_parquet(path)

In [65]:
import pathlib

path: pathlib.Path =  "./data/train_data_2/cleaned.parquet"

path

'./data/train_data_2/cleaned.parquet'